# NYC Energy Efficiency

In [1]:
import PyPDF2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [12]:
pdfFileObj = open('./NYC Energy Ratings.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
numPages = pdfReader.getNumPages()

In [13]:
# https://stackoverflow.com/questions/46705546/python-replace-every-nth-occurrence-of-string
def nth_repl_all(s, sub, repl, nth):
    find = s.find(sub)
    # loop util we find no match
    i = 1
    while find != -1:
        # if i  is equal to nth we found nth matches so replace
        if i == nth:
            s = s[:find]+repl+s[find + len(sub):]
            i = 0
        # find + len(sub) + 1 means we start after the last match
        find = s.find(sub, find + len(sub) + 1)
        i += 1
    return s

In [29]:
f = open("./ratings.txt", "a")

for page in range(numPages): #numPages
        pageObj = pdfReader.getPage(page)

        try:
            # exclude beginning and end jargon
            # also replace all newline characters with tabspaces
            str = pageObj.extractText().replace('\n', '\t')
            str = re.sub('\\tnyc.gov/buildings\\t\d* \\tof 710\\t', '\n',str)
            str = str.replace('LOCAL LAW 33 of 2018: \tData Disclosure 2019 CBL\t10 Digit BBL \tStreet Number\tStreet Name\tDOF Square \tFootage \tEnergy Star        \t1-100 Score\tEnergy Efficiency \tGrade\t', '')
            
            # drop all lines with missing data
            str = re.sub('\d{10}\\t\d*\\t.+\\tmissing required \tBenchmarking \tinformation\\t\w\\t','',str)
            
            f.write(nth_repl_all(str, '\t', '\n', 6))
            
            
        except:
            print('Error on page', str(page))
            
f.close()

In [2]:
df = pd.read_excel('./ratings.xlsx', index_col='BBL')

In [4]:
df

,street_address,street,sq_footage,score,rating
BBL,,,,,
1000080051,6,WATER STREET,102407,25,D
1000087501,2,WATER STREET,169061,13,D
1000090001,34,WHITEHALL STREET,692431,72,B
1000090014,17,STATE STREET,544015,73,B
1000090029,24,STATE STREET,896956,52,D
...,...,...,...,...,...
3073690027,2190,BRIGHAM STREET,167760,46,D
3083410100,1089,EAST 73 STREET,27975,89,A
3083410113,1073,EAST 73 STREET,27975,89,A


In [5]:
peri = pd.read_json('./bldg-perimeter.geojson')

In [11]:
import json
data = json.load(open('./bldg-perimeter.geojson'))
data['features'] #[0]['geometry'] #Your first point

[{'type': 'Feature',
  'properties': {'name': None,
   'base_bbl': '3044520815',
   'shape_area': '854.66243317866',
   'heightroof': '21.60850812',
   'mpluto_bbl': '3044520815',
   'cnstrct_yr': '2009',
   'lststatype': 'Constructed',
   'feat_code': '2100',
   'groundelev': '18',
   'geomsource': 'Photogramm',
   'bin': '3394646',
   'lstmoddate': '2017-08-22T00:00:00.000Z',
   'doitt_id': '1212853',
   'shape_len': '125.0797955584'},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[-73.87129515296562, 40.65717370043455],
      [-73.87135858020156, 40.65714663518705],
      [-73.87143322008981, 40.6572480836196],
      [-73.87136979278591, 40.6572751498085],
      [-73.87129515296562, 40.65717370043455]]]]}},
 {'type': 'Feature',
  'properties': {'name': None,
   'base_bbl': '4030640041',
   'shape_area': '217.59424346169',
   'heightroof': '10.36',
   'mpluto_bbl': '4030640041',
   'cnstrct_yr': '1930',
   'lststatype': 'Constructed',
   'feat_code': '5110',
   'grounde

In [37]:
peri = pd.DataFrame(data['features'])
peri

,type,properties,geometry
0,Feature,"{'name': None, 'base_bbl': '3044520815', 'shap...","{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
1,Feature,"{'name': None, 'base_bbl': '4030640041', 'shap...","{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,Feature,"{'name': None, 'base_bbl': '4139430001', 'shap...","{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
3,Feature,"{'name': None, 'base_bbl': '3049720006', 'shap...","{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
4,Feature,"{'name': None, 'base_bbl': '3055100055', 'shap...","{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
...,...,...,...
1124272,Feature,"{'name': None, 'base_bbl': '4069440038', 'heig...","{'type': 'Point', 'coordinates': [-73.79835625..."
1124273,Feature,"{'name': None, 'base_bbl': '4013210010', 'heig...","{'type': 'Point', 'coordinates': [-73.91240312..."
1124274,Feature,"{'name': None, 'base_bbl': '4005710020', 'heig...","{'type': 'Point', 'coordinates': [-73.92749543..."
1124275,Feature,"{'name': None, 'base_bbl': '4011300062', 'heig...","{'type': 'Point', 'coordinates': [-73.88693655..."


In [38]:
bldg_metadata = peri['properties'].apply(pd.Series)

In [39]:
bldg_metadata = bldg_metadata.join(peri['geometry'])

In [40]:
bldg_metadata = bldg_metadata[bldg_metadata['base_bbl'].notna()]

In [42]:
bldg_metadata

,name,base_bbl,shape_area,heightroof,mpluto_bbl,cnstrct_yr,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,shape_len,geometry
0,None,3044520815,854.66243317866,21.60850812,3044520815,2009,Constructed,2100,18,Photogramm,3394646,2017-08-22T00:00:00.000Z,1212853,125.0797955584,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
1,None,4030640041,217.59424346169,10.36,4030640041,1930,Constructed,5110,122,Photogramm,4548330,2017-08-17T00:00:00.000Z,1226227,60.22585821856,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,None,4139430001,946.42747637737,29.81157033,4139430001,1960,Constructed,2100,10,Photogramm,4460479,2017-08-22T00:00:00.000Z,581946,123.14194057237,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
3,None,3049720006,248.67816852809,11.2,3049720006,1920,Constructed,5110,32,Photogramm,3355684,2017-08-17T00:00:00.000Z,858061,63.94081721089,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
4,None,3055100055,1163.227668698,24.98,3055100055,1915,Constructed,2100,44,Photogramm,3131737,2017-08-22T00:00:00.000Z,568078,165.60876340496,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124272,None,4069440038,NaN,17.63,4069440038,1950,Constructed,2100,57,Photogramm,4150054,2017-08-22T00:00:00.000Z,834897,NaN,"{'type': 'Point', 'coordinates': [-73.79835625..."
1124273,None,4013210010,NaN,23.52,4013210010,1931,Constructed,2100,102,Photogramm,4030863,2017-08-22T00:00:00.000Z,772128,NaN,"{'type': 'Point', 'coordinates': [-73.91240312..."
1124274,None,4005710020,NaN,33.06,4005710020,1925,Constructed,2100,17,Photogramm,4006739,2017-08-22T00:00:00.000Z,729631,NaN,"{'type': 'Point', 'coordinates': [-73.92749543..."
1124275,None,4011300062,NaN,20.63,4011300062,1960,Constructed,2100,29,Photogramm,4025161,2017-08-22T00:00:00.000Z,448185,NaN,"{'type': 'Point', 'coordinates': [-73.88693655..."


In [43]:
bldg_metadata = bldg_metadata.set_index('base_bbl')

In [44]:
del bldg_metadata['name']
del bldg_metadata['geomsource']
del bldg_metadata['lstmoddate']
bldg_metadata

,shape_area,heightroof,mpluto_bbl,cnstrct_yr,lststatype,feat_code,groundelev,bin,doitt_id,shape_len,geometry
base_bbl,,,,,,,,,,,
3044520815,854.66243317866,21.60850812,3044520815,2009,Constructed,2100,18,3394646,1212853,125.0797955584,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
4030640041,217.59424346169,10.36,4030640041,1930,Constructed,5110,122,4548330,1226227,60.22585821856,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
4139430001,946.42747637737,29.81157033,4139430001,1960,Constructed,2100,10,4460479,581946,123.14194057237,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
3049720006,248.67816852809,11.2,3049720006,1920,Constructed,5110,32,3355684,858061,63.94081721089,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
3055100055,1163.227668698,24.98,3055100055,1915,Constructed,2100,44,3131737,568078,165.60876340496,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
...,...,...,...,...,...,...,...,...,...,...,...
4069440038,NaN,17.63,4069440038,1950,Constructed,2100,57,4150054,834897,NaN,"{'type': 'Point', 'coordinates': [-73.79835625..."
4013210010,NaN,23.52,4013210010,1931,Constructed,2100,102,4030863,772128,NaN,"{'type': 'Point', 'coordinates': [-73.91240312..."
4005710020,NaN,33.06,4005710020,1925,Constructed,2100,17,4006739,729631,NaN,"{'type': 'Point', 'coordinates': [-73.92749543..."


In [45]:
to_add = bldg_metadata.join(df)[bldg_metadata.join(df)['rating'].notna()]

In [51]:
to_add = to_add[['shape_area', 'heightroof', 'mpluto_bbl', 'cnstrct_yr', 'lststatype',
       'feat_code', 'groundelev', 'bin', 'doitt_id', 'shape_len', 'street_address', 'street', 'sq_footage', 'score', 'rating', 'geometry']]

In [54]:
df3 = to_add[~to_add.index.duplicated(keep='first')]
df3.to_json('./final.geojson')

In [56]:
df3['type'] = 'Feature'
df3

C:\Users\Vivek Rao\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,shape_area,heightroof,mpluto_bbl,cnstrct_yr,lststatype,feat_code,groundelev,bin,doitt_id,shape_len,street_address,street,sq_footage,score,rating,geometry,type
1000080051,4586.66704947048,None,1000080051,None,None,2100,79,1090472,1284733,317.85562148941,6,WATER STREET,102407,25,D,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
1000090001,21180.4720088324,442.25,1000090001,1970,Constructed,2100,6,1000018,406519,604.40984020387,34,WHITEHALL STREET,692431,72,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
1000090014,12196.7219173477,547.93,1000090014,1987,Constructed,2100,9,1000020,552070,438.93249630943,17,STATE STREET,544015,73,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
1000090029,22600.8010426496,446.23,1000090029,1971,Constructed,2100,12,1000021,230134,628.44151200673,24,STATE STREET,896956,52,D,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
1000100014,13660.7953267419,422.77,1000100014,1986,Constructed,2100,10,1000023,384935,487.80255351313,33,WHITEHALL STREET,365792,60,C,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5075840135,15799.0746670844,27,5075840135,2004,Constructed,2100,44,5155380,1091302,584.88055210864,30,SOUTH BRIDGE STREET,28334,88,A,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
5076260001,23467.1874204185,30.05334561,5076260001,1975,Constructed,2100,27,5087234,106097,667.07426082078,2,ARTHUR KILL ROAD,31300,82,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
5077020001,12751.5194771462,15.82,5077020001,1993,Constructed,2100,29,5110511,32204,481.37357877084,6420,AMBOY ROAD,69575,73,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
5079710250,8907.01960138525,16.06923283,5079710250,1963,Constructed,2100,21,5107861,887393,389.35042857168,236,RICHMOND VALLEY ROAD,69870,89,A,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature


In [60]:
df3.reset_index(inplace=True)
df3 = df3.rename(columns = {'index':'base_bbl'})

,base_bbl,shape_area,heightroof,mpluto_bbl,cnstrct_yr,lststatype,feat_code,groundelev,bin,doitt_id,shape_len,street_address,street,sq_footage,score,rating,geometry,type
0,1000080051,4586.66704947048,None,1000080051,None,None,2100,79,1090472,1284733,317.85562148941,6,WATER STREET,102407,25,D,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
1,1000090001,21180.4720088324,442.25,1000090001,1970,Constructed,2100,6,1000018,406519,604.40984020387,34,WHITEHALL STREET,692431,72,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
2,1000090014,12196.7219173477,547.93,1000090014,1987,Constructed,2100,9,1000020,552070,438.93249630943,17,STATE STREET,544015,73,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
3,1000090029,22600.8010426496,446.23,1000090029,1971,Constructed,2100,12,1000021,230134,628.44151200673,24,STATE STREET,896956,52,D,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
4,1000100014,13660.7953267419,422.77,1000100014,1986,Constructed,2100,10,1000023,384935,487.80255351313,33,WHITEHALL STREET,365792,60,C,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8102,5075840135,15799.0746670844,27,5075840135,2004,Constructed,2100,44,5155380,1091302,584.88055210864,30,SOUTH BRIDGE STREET,28334,88,A,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
8103,5076260001,23467.1874204185,30.05334561,5076260001,1975,Constructed,2100,27,5087234,106097,667.07426082078,2,ARTHUR KILL ROAD,31300,82,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
8104,5077020001,12751.5194771462,15.82,5077020001,1993,Constructed,2100,29,5110511,32204,481.37357877084,6420,AMBOY ROAD,69575,73,B,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature
8105,5079710250,8907.01960138525,16.06923283,5079710250,1963,Constructed,2100,21,5107861,887393,389.35042857168,236,RICHMOND VALLEY ROAD,69870,89,A,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Feature


In [58]:
df3['properties'] = dict(zip(df3.cnstrct_yr, df3.groundelev, df3.street_address, df3.street, df3.sq_footage, df3.score, df3.rating))

TypeError: dict expected at most 1 arguments, got 7

In [69]:
# https://geoffboeing.com/2015/10/exporting-python-data-geojson/

def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{}}
        feature['geometry'] = row['geometry']
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [70]:
df3.columns

Index(['base_bbl', 'shape_area', 'heightroof', 'mpluto_bbl', 'cnstrct_yr',
       'lststatype', 'feat_code', 'groundelev', 'bin', 'doitt_id', 'shape_len',
       'street_address', 'street', 'sq_footage', 'score', 'rating', 'geometry',
       'type'],
      dtype='object')

In [71]:
cols = ['base_bbl', 'cnstrct_yr',
       'groundelev', 'street_address', 'street', 'sq_footage', 'score', 'rating']

In [74]:
done_deal = df_to_geojson(df3, cols)

In [73]:
done_deal

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': None,
    'base_bbl': '3044520815',
    'shape_area': '854.66243317866',
    'heightroof': '21.60850812',
    'mpluto_bbl': '3044520815',
    'cnstrct_yr': '2009',
    'lststatype': 'Constructed',
    'feat_code': '2100',
    'groundelev': '18',
    'geomsource': 'Photogramm',
    'bin': '3394646',
    'lstmoddate': '2017-08-22T00:00:00.000Z',
    'doitt_id': '1212853',
    'shape_len': '125.0797955584'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-73.87129515296562, 40.65717370043455],
       [-73.87135858020156, 40.65714663518705],
       [-73.87143322008981, 40.6572480836196],
       [-73.87136979278591, 40.6572751498085],
       [-73.87129515296562, 40.65717370043455]]]]}},
  {'type': 'Feature',
   'properties': {'name': None,
    'base_bbl': '4030640041',
    'shape_area': '217.59424346169',
    'heightroof': '10.36',
    'mpluto_bbl': '4030640041',
    'cnstrct_yr': '1930

In [78]:
with open('./done_deal.txt', 'w') as convert_file:
     convert_file.write(json.dumps(done_deal))

f.close()